<img src="files/model_1_outline.png">

In [1]:
## change log
# Epoch to 100 epochs

sequence generator shape (corrected) (b, n, w, h, c)

## Changes: 
### Use camera 2
### Trim to 2 seconds

In [2]:
import os
import glob
import keras

In [3]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard
import tensorflow as tf
import datetime

In [4]:
# from patrice's blogpost
from keras_video import VideoFrameGenerator

In [5]:
classes = ['suture_throws', 'hand_ties', 'thread_cuts']
classes.sort()
print(classes)

['hand_ties', 'suture_throws', 'thread_cuts']


In [6]:
# some global params
SIZE = (100, 100) # height and width of frame pxl by pxl
CHANNELS = 3 # RGB or whatever
NBFRAME = 5 # num frames in sequence 
BS = 8 # Batch size

In [7]:
# pattern to get videos and classes
glob_pattern='../data_v3_model_2.2/model_data/{classname}/*.avi'

In [8]:
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)

In [9]:
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.33, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    use_frame_cache=True)

class hand_ties, validation count: 121, train count: 247
class suture_throws, validation count: 122, train count: 248
class thread_cuts, validation count: 121, train count: 248
Total data: 3 classes for 743 files for train


In [11]:
# getting validation data
valid = train.get_validation_generator()

Total data: 3 classes for 364 files for validation


In [11]:
# import keras_video.utils
# keras_video.utils.show_sample(train)

In [13]:
valid.files

['../data_v3_model_2.2/model_data/hand_ties\\hand_tie19.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie285.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie32.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie67.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie235.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie189.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie306.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie81.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie217.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie134.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie115.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie352.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie125.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie218.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie63.avi',
 '../data_v3_model_2.2/model_data/hand_ties\\hand_tie164.avi

## BUILD CONV NET

In [12]:
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
def build_mobilenet(shape=(224, 224, 3), nbout=3):
    model = keras.applications.mobilenet.MobileNet(
        include_top=False,
        input_shape=shape,
        weights='imagenet')
    # Keep 9 layers to train﻿﻿
    trainable = 9
    for layer in model.layers[:-trainable]:
        layer.trainable = False
    for layer in model.layers[-trainable:]:
        layer.trainable = True
    output = GlobalMaxPool2D()
    return keras.Sequential([model, output])

## Build GRU

In [13]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout
# Shape (5, 112, 112, 3) 5 - time sequence length 112x112 = height vs width 3 - num channels
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_mobilenet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    # KEY = allows you to add a time sequence to a layer one at a time
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

## this is where you tell the model how to train - loss function, weight update mechanism

In [14]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 112, 112, 3)
# action model - GRU set up for Time shifted CNN
model = action_model(INSHAPE, len(classes))

# this is where you tell the model how to train - loss function, weight update mechanism
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

## epochs, call backs

In [15]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
print('logs for this run are here: {}'.format(log_dir))

logs for this run are here: logs/fit/20210406-144120


In [16]:
EPOCHS=100
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'model2_2_chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
    tensorboard_callback
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/100
92/92 [==============================] - 364s 4s/step - loss: 1.0992 - acc: 0.3971 - val_loss: 1.0063 - val_acc: 0.5361

Epoch 00001: saving model to model2_2_chkp\weights.01-1.01.hdf5
Epoch 2/100
92/92 [==============================] - 27s 290ms/step - loss: 0.9060 - acc: 0.5748 - val_loss: 0.8158 - val_acc: 0.6222

Epoch 00002: saving model to model2_2_chkp\weights.02-0.82.hdf5
Epoch 3/100
92/92 [==============================] - 24s 260ms/step - loss: 0.7791 - acc: 0.6531 - val_loss: 0.8065 - val_acc: 0.6250

Epoch 00003: saving model to model2_2_chkp\weights.03-0.81.hdf5
Epoch 4/100
92/92 [==============================] - 24s 261ms/step - loss: 0.7672 - acc: 0.6497 - val_loss: 0.6254 - val_acc: 0.7861

Epoch 00004: saving model to model2_2_chkp\weights.04-0.63.hdf5
Epoch 5/100
92/92 [==============================] - 25s 271ms/step - loss: 0.5971 - acc: 0.7656 - val_loss: 0.6518 - val_acc: 0.7389

Epoch 00005: saving model to model2_2_chkp\weights.05-0.65.hdf5
Epoch 6


Epoch 00041: saving model to model2_2_chkp\weights.41-0.36.hdf5
Epoch 42/100
92/92 [==============================] - 24s 262ms/step - loss: 0.0549 - acc: 0.9890 - val_loss: 0.3746 - val_acc: 0.9167

Epoch 00042: saving model to model2_2_chkp\weights.42-0.37.hdf5
Epoch 43/100
92/92 [==============================] - 25s 270ms/step - loss: 0.0638 - acc: 0.9837 - val_loss: 0.4040 - val_acc: 0.9139

Epoch 00043: saving model to model2_2_chkp\weights.43-0.40.hdf5
Epoch 44/100
92/92 [==============================] - 24s 266ms/step - loss: 0.0598 - acc: 0.9823 - val_loss: 0.4397 - val_acc: 0.9167

Epoch 00044: saving model to model2_2_chkp\weights.44-0.44.hdf5
Epoch 45/100
92/92 [==============================] - 25s 267ms/step - loss: 0.0670 - acc: 0.9762 - val_loss: 0.4083 - val_acc: 0.9083

Epoch 00045: saving model to model2_2_chkp\weights.45-0.41.hdf5
Epoch 46/100
92/92 [==============================] - 25s 270ms/step - loss: 0.1031 - acc: 0.9686 - val_loss: 0.3517 - val_acc: 0.9083


92/92 [==============================] - 24s 267ms/step - loss: 0.0601 - acc: 0.9784 - val_loss: 0.4022 - val_acc: 0.9167

Epoch 00081: saving model to model2_2_chkp\weights.81-0.40.hdf5
Epoch 82/100
92/92 [==============================] - 25s 269ms/step - loss: 0.0262 - acc: 0.9934 - val_loss: 0.3980 - val_acc: 0.9167

Epoch 00082: saving model to model2_2_chkp\weights.82-0.40.hdf5
Epoch 83/100
92/92 [==============================] - 25s 268ms/step - loss: 0.0295 - acc: 0.9896 - val_loss: 0.3817 - val_acc: 0.9222

Epoch 00083: saving model to model2_2_chkp\weights.83-0.38.hdf5
Epoch 84/100
92/92 [==============================] - 24s 262ms/step - loss: 0.0266 - acc: 0.9900 - val_loss: 0.3927 - val_acc: 0.9194

Epoch 00084: saving model to model2_2_chkp\weights.84-0.39.hdf5
Epoch 85/100
92/92 [==============================] - 24s 266ms/step - loss: 0.0852 - acc: 0.9657 - val_loss: 0.3977 - val_acc: 0.9167

Epoch 00085: saving model to model2_2_chkp\weights.85-0.40.hdf5
Epoch 86/100
